In [ ]:
import pickle
from scipy.stats import permutation_test
from scipy.stats import bootstrap
import numpy as np
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
from io import BytesIO
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt 

In [ ]:
!mkdir -p dev/results
!wget -O dev/results/de_novo_results.zip \
    "https://zenodo.org/records/16438770/files/de_novo_results.zip?download=1"
!unzip -o dev/results/de_novo_results.zip -d dev/results/
!rm dev/results/de_novo_results.zip

!mkdir -p dev/results
!wget -O dev/results/fragmentation_results.zip \
    "https://zenodo.org/records/16438770/files/fragmentation_results.zip?download=1"
!unzip -o dev/results/fragmentation_results.zip -d dev/results/
!rm dev/results/fragmentation_results.zip

In [ ]:
base_folder = Path("dev/results")

In [ ]:
df = (
    pd.concat(        
    [
            pickle.load(
                BytesIO(p.read_bytes())
            ).assign(
                fragments=True,
            )
            for p in (base_folder / "fragmentation_results").iterdir()
        ] + [
            pickle.load(
                BytesIO(p.read_bytes())
            ).assign(
                fragments=False,
            )
            for p in (base_folder / "de_novo_results").iterdir()
        ], ignore_index=True
    )
).loc[
    lambda d: d['model'].eq("seismiq_pretrained")
]

df

In [ ]:
df_acc = (
    df.groupby(
        ["model", "dataset", "challenge", "fragments", "bond_idx", "dummy_idx", "given_atoms", "missing_atoms"],
        dropna=False,
    )['tanimoto']
    .max()
    .to_frame()
    .reset_index()
    .assign(
        accuracy=lambda d: d["tanimoto"] > 0.9999,
        missing_atoms_bin=lambda d: pd.cut(
            d["missing_atoms"],
            bins=[1, 5, 10, 20, 30, 45, 90, 9000]
        )
    )
)

df_acc

In [ ]:
results = []
groups = df_acc.groupby(
    ["model", "dataset", "missing_atoms_bin"],
    dropna=False,
)
for (model, dataset, missing), group in groups:
    vals = group["accuracy"].values.astype(int)
    res = bootstrap(
        (vals,),
        statistic=lambda x: np.mean(x),
        alternative="two-sided",
    )

    ci_low, ci_high = res.confidence_interval.low, res.confidence_interval.high
    if res.standard_error < 1e-6:
        # these are nan's if the values are all the same
        ci_low = ci_high = np.mean(vals)

    results.append(
        {
            #"model": model,
            "dataset": dataset,
            "missing": missing,
            "correct": vals.sum(),
            "total": len(vals),
            "accuracy": vals.mean(),
            "ci_low": ci_low,
            "ci_high": ci_high,
            "se": res.standard_error,
        }
    )

In [ ]:
df_res = (
    pd.DataFrame(results)
    .assign(
        missing=lambda d: np.where(d["missing"].isna(), "(all)", d["missing"].astype(str)),
    )
    .set_index(["dataset", "missing"])
)

from IPython.display import display
with pd.option_context('display.float_format', '{:.4f}'.format):
    display(df_res)